## TPC-H BigQuery Import  
Import data from GCS to a previously created BigQuery dataset

In [ ]:
import config, schema, bq, load

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option("display.max_rows", 1000)
pd.options.display.float_format = '{:.2f}'.format

### I. TPC-H 1GB - basic, no config

In [ ]:
dataset_name = 'h_1GB_qual'

In [ ]:
ddl_filepath = config.fp_schema + config.sep + "bq_h.ddl"

In [ ]:
bq.create_dataset(dataset_name=dataset_name)

In [ ]:
bq.create_schema(schema_file=ddl_filepath, dataset=dataset_name)

In [ ]:
u = load.BQUpload(dataset_name=dataset_name,
                  test="h", scale=1)
u.verbose = True
u.inventory_bucket()
u.subset_df()

In [ ]:
u.df

In [ ]:
results = u.upload()

In [ ]:
u.df

In [ ]:
# TODO: remake Pooled Upload to break tables out to individual threads and upload sequentially from there

In [ ]:
dfx = pd.concat(u.up_data)
dfx.sort_index(inplace=True)

In [ ]:
plt.plot(u.df.index, '+',
         markersize=10, 
         markerfacecolor=None,
         markeredgecolor='blue');
plt.plot(dfx.index, 'x',
         markersize=10, 
         markerfacecolor=None,
         markeredgecolor='green');

In [ ]:
dfx[["chunk_name", "size_bytes", "table", "done", "MB/s"]]

### II. TPC-H 2GB - basic, no config

In [ ]:
break

In [ ]:
bq.create_schema(schema_file=ddl_filepath)

In [ ]:
# set n to about 5, high numbers run afoul of BQ's upload limits
# i.e. 403 - rateLimitExceeded
# https://cloud.google.com/bigquery/docs/error-messages
u = load.BQPooledUpload(dataset_name=dataset_name,
                        test="h", scale=2, n=5)
u.verbose = True

In [ ]:
results = u.pipeline()

In [ ]:
a = list(results)
a

In [ ]:
dfx = pd.concat(u.up_data)
dfx.sort_index(inplace=True)

In [ ]:
plt.plot(u.df.index, '+',
         markersize=10, 
         markerfacecolor=None,
         markeredgecolor='blue');
plt.plot(dfx.index, 'x',
         markersize=10, 
         markerfacecolor=None,
         markeredgecolor='green');

In [ ]:
dfx[["chunk_name", "size_bytes", "table", "done", "MB/s"]]

In [ ]:
break

### II. Validate Data Uploaded  

In [ ]:
reload(load)
valid_up = load.BQUpload(dataset_name=dataset_name, 
                        test="h", 
                        scale=1)

In [ ]:
valid_up.inventory_bucket()

In [ ]:
valid_up.compile_df()

In [ ]:
valid_up.inventory_data